### Indepth Analysis

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
sns.set()

In [2]:
application_train = pd.read_csv('new_application_train_merged.csv', index_col=None, engine='python')

In [3]:
application_test = pd.read_csv('new_application_test_merged.csv', index_col=None, engine='python')

In [4]:
new_application_train = pd.read_csv('application_train_for_ML.csv', index_col=None, engine='python')

In [5]:
new_application_test = pd.read_csv('application_test_for_ML.csv', index_col=None, engine='python')

## Feature Imputing & Feature Scaling 


In [6]:
new_application_train.shape

(291057, 20)

In [7]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,classification_report, roc_auc_score

In [8]:
imputer = SimpleImputer(strategy="median")
scaler = MinMaxScaler(feature_range = (0, 1))

In [9]:
imputer.fit(new_application_train)


SimpleImputer(strategy='median')

In [10]:
train_transformed = imputer.transform(new_application_train)


In [11]:
test_transformed = imputer.transform(new_application_test)


In [12]:
train_transformed = scaler.fit_transform(train_transformed)


In [13]:
test_transformed = scaler.fit_transform(test_transformed)


In [14]:
X_training_set, X_validation_set, y_training_set, y_validation_set = train_test_split(train_transformed, 
                                                                                      application_train['TARGET'], test_size=0.33, random_state=42)

## 1. Logistic Regression

In [15]:
from sklearn.linear_model import LogisticRegression
logistic_regressor = LogisticRegression(C = 2)
logistic_regressor.fit(X_training_set,y_training_set)
log_regression_pred = logistic_regressor.predict(X_validation_set)


C:\Users\kneelagandan\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [16]:
log_regression_pred

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [17]:
pd.DataFrame({'target':log_regression_pred})['target'].value_counts()


0    95939
1      110
Name: target, dtype: int64

In [18]:
from sklearn.metrics import accuracy_score,classification_report, roc_auc_score


In [19]:
print("The accuracy score : ", accuracy_score(y_validation_set,log_regression_pred))
print("\n")
print("The classification report is as follows:\n", classification_report(y_validation_set,log_regression_pred))
print("\n")
print("ROC AUC score is: ",roc_auc_score(y_validation_set,log_regression_pred))

The accuracy score :  0.9179168965840352


The classification report is as follows:
               precision    recall  f1-score   support

           0       0.92      1.00      0.96     88165
           1       0.50      0.01      0.01      7884

    accuracy                           0.92     96049
   macro avg       0.71      0.50      0.49     96049
weighted avg       0.88      0.92      0.88     96049



ROC AUC score is:  0.5031761619591373


In [20]:
log_regression_pred_test = logistic_regressor.predict_proba(test_transformed)
log_regression_pred_test[:,1]


array([0.04810939, 0.14203743, 0.03369764, ..., 0.08638704, 0.05137397,
       0.16553288])

In [21]:
submission_log_regression = application_test[['SK_ID_CURR']]
submission_log_regression['TARGET'] = log_regression_pred_test[:,1]

<ipython-input-21-e110191343b8>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submission_log_regression['TARGET'] = log_regression_pred_test[:,1]


## 2. Random Forest 

In [22]:
from sklearn.ensemble import RandomForestClassifier

random_forest = RandomForestClassifier(n_estimators = 500, random_state = 50, verbose = 1)

In [23]:
random_forest.fit(X_training_set,y_training_set)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:  3.5min finished


RandomForestClassifier(n_estimators=500, random_state=50, verbose=1)

In [24]:
random_forest_pred = random_forest.predict(X_validation_set)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:   14.7s finished


In [25]:
from sklearn.metrics import accuracy_score,classification_report,roc_auc_score
print("The accuracy in general is : ", accuracy_score(y_validation_set,random_forest_pred))
print("\n")
print("The classification report is as follows:\n", classification_report(y_validation_set,random_forest_pred))
print("ROC AUC score is: ",roc_auc_score(y_validation_set,random_forest_pred))

The accuracy in general is :  0.9182500598652771


The classification report is as follows:
               precision    recall  f1-score   support

           0       0.92      1.00      0.96     88165
           1       0.60      0.01      0.02      7884

    accuracy                           0.92     96049
   macro avg       0.76      0.51      0.49     96049
weighted avg       0.89      0.92      0.88     96049

ROC AUC score is:  0.5056675758401546


In [26]:
random_forest_pred_test = random_forest.predict_proba(test_transformed)
submission_random_forest = application_test[['SK_ID_CURR']]
submission_random_forest['TARGET'] = random_forest_pred_test[:,1]

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:    6.7s finished
<ipython-input-26-86f9992d35cb>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submission_random_forest['TARGET'] = random_forest_pred_test[:,1]


## Feature importance of random forest classifier & top 20 important features based on Algorithm


In [27]:
features = list(set(new_application_train.columns).difference({'TARGET'}))

feature_importance_df = pd.DataFrame({'Feature':features,'Importance':random_forest.feature_importances_})
df = feature_importance_df.sort_values(['Importance'],ascending=False).reset_index()
df = df.drop(['index'],axis=1)
df.head(20)

,Feature,Importance
0,REGION_RATING_CLIENT,0.176560
1,EXT_SOURCE_2,0.142101
2,AMT_CREDIT_x,0.140546
3,EXT_SOURCE_1,0.128994
4,NAME_FAMILY_STATUS,0.084314
5,AMT_ANNUITY_x_1,0.070825
6,CNT_FAM_MEMBERS,0.058255
7,ORGANIZATION_TYPE,0.038416
8,OCCUPATION_TYPE,0.033162
9,NAME_HOUSING_TYPE,0.022051


## 3. Naive Bayes Classifier


In [28]:
from sklearn.naive_bayes import GaussianNB


In [29]:
bayes_class = GaussianNB()


In [30]:
bayes_class.fit(X_training_set,y_training_set)


GaussianNB()

In [31]:
bayes_preds = bayes_class.predict(X_validation_set)


In [32]:
pd.DataFrame({'target':bayes_preds})['target'].value_counts()


0    89637
1     6412
Name: target, dtype: int64

In [33]:
from sklearn.metrics import accuracy_score,classification_report,roc_auc_score
print("The accuracy in general is : ", accuracy_score(y_validation_set,bayes_preds))
print("\n")
print("The classification report is as follows:\n", classification_report(y_validation_set,bayes_preds))
print("ROC AUC score is: ",roc_auc_score(y_validation_set,bayes_preds))

The accuracy in general is :  0.8817062124540599


The classification report is as follows:
               precision    recall  f1-score   support

           0       0.93      0.94      0.94     88165
           1       0.23      0.19      0.21      7884

    accuracy                           0.88     96049
   macro avg       0.58      0.56      0.57     96049
weighted avg       0.87      0.88      0.88     96049

ROC AUC score is:  0.5649925212870118


In [34]:
nb_pred_test = bayes_class.predict_proba(test_transformed)
submission_nb = application_test[['SK_ID_CURR']]
submission_nb['TARGET'] = nb_pred_test[:,1]


<ipython-input-34-3115073d959d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submission_nb['TARGET'] = nb_pred_test[:,1]


## 4 . Ensemble Modeling 
https://quantdare.com/what-is-the-difference-between-bagging-and-boosting/

In [35]:
from scipy import stats
#Voting Approach
def stacked_model(X_training_set):
    """
    This method performs the stacked ensambling of all the models - Random forest, 
    Naive Bayes,Logistic Regression.
    """  
    stacked_predictions = np.array([])

    for element in X_training_set:
         stacked_predictions = np.append(stacked_predictions,stats.mode(element)[0][0])

    return stacked_predictions

Combine all the test results into a multidimensional array to feed into the stacked model

In [36]:
combined_array = (pd.DataFrame({'LR':log_regression_pred,
                                'RF':random_forest_pred,
                                'Bayes':bayes_preds}).values)

In [37]:
stacked_model_pred = stacked_model(combined_array)
pd.DataFrame({'target':stacked_model_pred})['target'].value_counts()


0.0    95839
1.0      210
Name: target, dtype: int64

In [38]:
print("The accuracy in general is : ", accuracy_score(y_validation_set,stacked_model_pred))
print("\n")
print("The classification report is as follows:\n", classification_report(y_validation_set,stacked_model_pred))
print("ROC AUC score is: ",roc_auc_score(y_validation_set,stacked_model_pred))

The accuracy in general is :  0.9181875917500443


The classification report is as follows:
               precision    recall  f1-score   support

           0       0.92      1.00      0.96     88165
           1       0.56      0.01      0.03      7884

    accuracy                           0.92     96049
   macro avg       0.74      0.51      0.49     96049
weighted avg       0.89      0.92      0.88     96049

ROC AUC score is:  0.5069617619148037


## 5 .XG Boost

In [39]:
from xgboost import XGBClassifier


In [40]:
xgb_classifier = XGBClassifier(n_estimators=250,max_depth=5)
xgb_classifier.fit(X_training_set,y_training_set)
xgb_pred = xgb_classifier.predict(X_validation_set)
pd.DataFrame({'target':xgb_pred})['target'].value_counts()


0    95603
1      446
Name: target, dtype: int64

In [41]:
from sklearn.metrics import accuracy_score,classification_report,roc_auc_score
print("The accuracy in general is : ", accuracy_score(y_validation_set,xgb_pred))
print("\n")
print("The classification report is as follows:\n", classification_report(y_validation_set,xgb_pred))
print("ROC AUC score is: ",roc_auc_score(y_validation_set,xgb_pred))

The accuracy in general is :  0.9177919603535696


The classification report is as follows:
               precision    recall  f1-score   support

           0       0.92      1.00      0.96     88165
           1       0.49      0.03      0.05      7884

    accuracy                           0.92     96049
   macro avg       0.70      0.51      0.50     96049
weighted avg       0.88      0.92      0.88     96049

ROC AUC score is:  0.5124633484222526


In [42]:
xgb_pred_test = xgb_classifier.predict_proba(test_transformed)
submission_xgb = application_test[['SK_ID_CURR']]
submission_xgb['TARGET'] = xgb_pred_test[:,1]


<ipython-input-42-65d003118767>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submission_xgb['TARGET'] = xgb_pred_test[:,1]


## Neural Network 

In [43]:
#nerual networks api
from keras.layers import Input, Dense
from keras.models import Model, Sequential
from keras import regularizers
from keras import optimizers
import tensorflow as tf

In [44]:
model = Sequential()
new_application_test.shape[1]
model.add(Dense(100, input_dim=new_application_train.shape[1], activation= "relu"))
model.add(Dense(50, activation= "relu"))
model.add(Dense(1,activation= "sigmoid"))
model.summary() #Print model Summary


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               2100      
_________________________________________________________________
dense_1 (Dense)              (None, 50)                5050      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 51        
Total params: 7,201
Trainable params: 7,201
Non-trainable params: 0
_________________________________________________________________


In [45]:
model.compile(loss= "binary_crossentropy" , optimizer="adam", metrics=["accuracy"])
model.fit(X_training_set, y_training_set, epochs=10)


Epoch 1/10
6094/6094 [==============================] - 6s 903us/step - loss: 0.2604 - accuracy: 0.9181
Epoch 2/10
6094/6094 [==============================] - 5s 799us/step - loss: 0.2584 - accuracy: 0.9182
Epoch 3/10
6094/6094 [==============================] - 5s 834us/step - loss: 0.2579 - accuracy: 0.9181
Epoch 4/10
6094/6094 [==============================] - 8s 1ms/step - loss: 0.2576 - accuracy: 0.9181
Epoch 5/10
6094/6094 [==============================] - 6s 1ms/step - loss: 0.2575 - accuracy: 0.9182
Epoch 6/10
6094/6094 [==============================] - 6s 904us/step - loss: 0.2574 - accuracy: 0.9182
Epoch 7/10
6094/6094 [==============================] - 6s 1ms/step - loss: 0.2573 - accuracy: 0.9182
Epoch 8/10
6094/6094 [==============================] - 7s 1ms/step - loss: 0.2572 - accuracy: 0.9182
Epoch 9/10
6094/6094 [==============================] - 8s 1ms/step - loss: 0.2568 - accuracy: 0.9182
Epoch 10/10
6094/6094 [==============================] - 6s 1ms/step - los

In [46]:
pred= model.predict(X_validation_set)


In [47]:
nn_pred_test = model.predict(test_transformed)
nn_pred_test_conv = [y[0] for y in nn_pred_test]


In [48]:
submission_xgb = application_test[['SK_ID_CURR']]
submission_xgb['TARGET'] = nn_pred_test_conv

<ipython-input-48-64d487924bf4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submission_xgb['TARGET'] = nn_pred_test_conv


## Hyper Parameter Tuning 

## Logistic Regression - Grid Search CV

In [49]:
from sklearn.linear_model import LogisticRegression
import numpy as np
from sklearn.model_selection import GridSearchCV 

logistic_regressor = LogisticRegression()

c_space = np.logspace(-5, 8, 15) 
param_grid = {'C': c_space} 

logreg_cv = GridSearchCV(logistic_regressor, param_grid, cv = 5) 
logreg_cv.fit(X_training_set,y_training_set)


log_regression_pred = logreg_cv.predict(X_validation_set)

C:\Users\kneelagandan\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\kneelagandan\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regres

C:\Users\kneelagandan\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\kneelagandan\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regres

In [50]:
from sklearn.metrics import accuracy_score,classification_report, roc_auc_score
print("The accuracy score : ", accuracy_score(y_validation_set,log_regression_pred))
print("\n")
print("The classification report is as follows:\n", classification_report(y_validation_set,log_regression_pred))
print("\n")
print("ROC AUC score is: ",roc_auc_score(y_validation_set,log_regression_pred))

The accuracy score :  0.9179064852314964


The classification report is as follows:
               precision    recall  f1-score   support

           0       0.92      1.00      0.96     88165
           1       0.50      0.01      0.01      7884

    accuracy                           0.92     96049
   macro avg       0.71      0.50      0.49     96049
weighted avg       0.88      0.92      0.88     96049



ROC AUC score is:  0.5031704907744269


In [51]:
log_regression_pred_test = logreg_cv.predict_proba(test_transformed)
log_regression_pred_test[:,1]
submission_log_regression = application_test[['SK_ID_CURR']]
submission_log_regression['TARGET'] = log_regression_pred_test[:,1]

<ipython-input-51-958f2f9d48fc>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submission_log_regression['TARGET'] = log_regression_pred_test[:,1]


In [52]:
submission_log_regression.head(10)


,SK_ID_CURR,TARGET
0,100001,0.047857
1,100005,0.142030
2,100013,0.034110
3,100028,0.048322
4,100038,0.148227
5,100042,0.059829
6,100057,0.020161
7,100065,0.060503
8,100066,0.018878
9,100067,0.192676


In [53]:
print("Best score is {}".format(logreg_cv.best_score_)) 


Best score is 0.9181982287776703


## Accuracy Score bit improved after tuning the C value

## XG Boost - Random Search CV

In [54]:
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV

params = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'max_depth': [3, 4, 5]
        }

In [55]:
xgb = XGBClassifier(learning_rate=0.02, n_estimators=600, objective='binary:logistic',
                    silent=True, nthread=1)

In [56]:
folds = 3
param_comb = 3

skf = StratifiedKFold(n_splits=folds, shuffle = True, random_state = 1001)

random_search = RandomizedSearchCV(xgb, param_distributions=params, n_iter=param_comb, scoring='roc_auc', n_jobs=4, cv=skf.split(X_training_set,y_training_set), verbose=3, random_state=1001 )

# Here we go
random_search.fit(X_training_set,y_training_set)


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


Fitting 3 folds for each of 3 candidates, totalling 9 fits


[Parallel(n_jobs=4)]: Done   6 out of   9 | elapsed:  4.6min remaining:  2.3min
[Parallel(n_jobs=4)]: Done   9 out of   9 | elapsed:  5.2min finished


[17:56:44] WARNING: C:\Users\Administrator\workspace\xgboost-win64_release_1.2.0\src\learner.cc:516: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




RandomizedSearchCV(cv=<generator object _BaseKFold.split at 0x000001D188D40120>,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, gamma=None,
                                           gpu_id=None, importance_type='gain',
                                           interaction_constraints=None,
                                           learning_rate=0.02,
                                           max_delta_step=None, max_depth=None,
                                           min_child_weight=None, mis...
                                           n_estimators=600, n_jobs=None,
                                           nthread=1, num_parallel_tree=None,
                                           random_state=None, reg_alpha=None,
                                   

In [57]:
print('Hi')

Hi


In [58]:
print(random_search.best_params_)

{'min_child_weight': 10, 'max_depth': 4, 'gamma': 2}


In [59]:
xgb_pred = random_search.predict(X_validation_set)


In [60]:
from sklearn.metrics import accuracy_score,classification_report,roc_auc_score
print("The accuracy in general is : ", accuracy_score(y_validation_set,xgb_pred))
print("\n")
print("The classification report is as follows:\n", classification_report(y_validation_set,xgb_pred))
print("ROC AUC score is: ",roc_auc_score(y_validation_set,xgb_pred))

The accuracy in general is :  0.9180522441670398


The classification report is as follows:
               precision    recall  f1-score   support

           0       0.92      1.00      0.96     88165
           1       0.55      0.01      0.02      7884

    accuracy                           0.92     96049
   macro avg       0.73      0.50      0.49     96049
weighted avg       0.89      0.92      0.88     96049

ROC AUC score is:  0.5045203521440286


In [61]:
xgb_pred_test = xgb_classifier.predict_proba(test_transformed)
submission_xgb = application_test[['SK_ID_CURR']]
submission_xgb['TARGET'] = xgb_pred_test[:,1]
submission_xgb.head(10)

<ipython-input-61-91ffe88df655>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submission_xgb['TARGET'] = xgb_pred_test[:,1]


,SK_ID_CURR,TARGET
0,100001,0.007640
1,100005,0.095211
2,100013,0.018692
3,100028,0.077636
4,100038,0.140996
5,100042,0.098866
6,100057,0.013254
7,100065,0.051671
8,100066,0.002262
9,100067,0.103773
